In [0]:
# configs = {"fs.azure.account.key.storage001de.blob.core.windows.net": "MA38y0ZA2Ij1L3QXl/F/Y8EGXR1VjTGb896PNbUmPT99uhVLoi0qCEiBEBagBQ4yscqwq9Zxb0vX+AStlr0Z3Q=="}

# # create mount to silver
# dbutils.fs.mount(
# 	source = "wasbs://silver@storage001de.blob.core.windows.net/",
# 	mount_point = "/mnt/datalake/silver",
# 	extra_configs = configs
# )

In [0]:
df_keyword = spark.read.table('bronze.keyword')
df_movie_keyword = spark.read.table('bronze.movie_to_keyword')


In [0]:
df_keyword.show(5)

+---+--------------------+--------------------+----------+------------------+
| id|          created_at|          updated_at|deleted_at|              name|
+---+--------------------+--------------------+----------+------------------+
|  1|2023-11-17 17:15:...|2023-11-17 17:15:...|      NULL|     new york city|
|  2|2023-11-17 17:15:...|2023-11-17 17:15:...|      NULL|love of one's life|
|  3|2023-11-17 17:15:...|2023-11-17 17:15:...|      NULL|       time travel|
|  4|2023-11-17 17:15:...|2023-11-17 17:15:...|      NULL|              duke|
|  5|2023-11-17 17:15:...|2023-11-17 17:15:...|      NULL| fish out of water|
+---+--------------------+--------------------+----------+------------------+
only showing top 5 rows



In [0]:
df_movie_keyword.show(5)

+-------+---------+
|movieId|keywordId|
+-------+---------+
|      1|        1|
|      7|        1|
|     31|        1|
|    108|        1|
|    148|        1|
+-------+---------+
only showing top 5 rows



In [0]:
df_keyword = df_keyword.dropDuplicates(['id'])
df_movie_keyword = df_movie_keyword.dropDuplicates()

#### Rename keywordId -> id, name -> keywords

In [0]:
df_movie_keyword = df_movie_keyword.withColumnRenamed('keywordId', 'id')
df_keyword = df_keyword.withColumnRenamed('name', 'keywords')

#### Left join

In [0]:
df_new = df_movie_keyword.join(df_keyword , on=['id'] , how = 'left')

In [0]:
df_new.show()

+---+-------+--------------------+--------------------+----------+-------------+
| id|movieId|          created_at|          updated_at|deleted_at|     keywords|
+---+-------+--------------------+--------------------+----------+-------------+
|  1|      1|2023-11-17 17:15:...|2023-11-17 17:15:...|      NULL|new york city|
|  1|      7|2023-11-17 17:15:...|2023-11-17 17:15:...|      NULL|new york city|
|  1|     31|2023-11-17 17:15:...|2023-11-17 17:15:...|      NULL|new york city|
|  1|    108|2023-11-17 17:15:...|2023-11-17 17:15:...|      NULL|new york city|
|  1|    148|2023-11-17 17:15:...|2023-11-17 17:15:...|      NULL|new york city|
|  1|    204|2023-11-17 17:15:...|2023-11-17 17:15:...|      NULL|new york city|
|  1|    235|2023-11-17 17:15:...|2023-11-17 17:15:...|      NULL|new york city|
|  1|    256|2023-11-17 17:15:...|2023-11-17 17:15:...|      NULL|new york city|
|  1|    296|2023-11-17 17:15:...|2023-11-17 17:15:...|      NULL|new york city|
|  1|    331|2023-11-17 17:1

In [0]:
df_new = df_new.select('movieId', 'keywords')

In [0]:
df_new.sort(df_new['movieId'].asc())
df_new.show(5)
df_new.count()

+-------+-------------+
|movieId|     keywords|
+-------+-------------+
|      1|new york city|
|      7|new york city|
|     31|new york city|
|    108|new york city|
|    148|new york city|
+-------+-------------+
only showing top 5 rows



396743

### drop duplicates

In [0]:
df_new = df_new.dropDuplicates(subset=['movieId', 'keywords'])
df_new.count()


396743

In [0]:
from pyspark.sql.functions import collect_list
df_new = df_new.groupBy("movieId").agg(collect_list("keywords").alias("keywords"))
df_new.show()

+-------+--------------------+
|movieId|            keywords|
+-------+--------------------+
|      1|[time portal, mar...|
|     12|[night life, love...|
|     13|[fraud, con man, ...|
|     22|[cinematographer,...|
|     26|[milan, work, wor...|
|     27|[insurgence, esca...|
|     28|[island, tree hou...|
|     31|[cigarette, write...|
|     34|[global warming, ...|
|     44|[puppy, dog, fema...|
|     47|[asylum, unrequit...|
|     52|[car crash, b mov...|
|     53|[cancer, algerien...|
|     65|[stepparents, ele...|
|     76|[adultery, scapeg...|
|     78|[ohio, songwriter...|
|     81|[war room, anti w...|
|     85|[disappearance, b...|
|     91|[homelessness, de...|
|     93|[cattle baron, ci...|
+-------+--------------------+
only showing top 20 rows



#### Save table

In [0]:
# Define the silver location
keyword_silverLocation = "/mnt/datalake/silver/cleaned_keyword"

# Table name
keyword_bronzeTable = "bronze.keyword"
keyword_silverTable = "silver.keyword"

In [0]:
from delta.tables import *
from pyspark.sql.functions import col, substring

# #check if the silver location contain the delta table
# if (DeltaTable.isDeltaTable(spark, movie_silverLocation)):
#     DeltaTable.forPath(spark, movie_silverLocation).alias("target").merge(
#         source = cleaned_movie.alias("src"),
#         condition = "target.id = src.id"
#     ).WhenNoMatchedInsertAll().execute()
# else:
df_new.write.mode("overwrite").format("delta").option("overwriteSchema", "true").save(keyword_silverLocation)
    

In [0]:
spark.sql(f"CREATE EXTERNAL TABLE IF NOT EXISTS {keyword_silverTable} USING delta LOCATION '/mnt/datalake/silver/cleaned_keyword'")

DataFrame[]

In [0]:
# Maintenance for Delta table

# To optimized the performance of the Delta table, we need to excute 2 commands:
# 1. optimize(): Optimize the number of files used to store the data
# 2. vacuum(): Remove the old version of the data. This reduces the overhead but it limites the version we can go back to.
keyword_DataDelta = DeltaTable.forName(spark, keyword_silverTable)

if keyword_DataDelta.history(30).filter("operation = 'VACUUM START'").count() == 0:
	keyword_DataDelta.optimize()
	keyword_DataDelta.vacuum()


In [0]:
%sql
DESCRIBE HISTORY silver.keyword;

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
2,2023-12-13T11:28:11Z,1311347395691033,totnvt02@gmail.com,VACUUM END,Map(status -> COMPLETED),null,List(1731309032100310),1122-230829-ksqo7xkx,1,SnapshotIsolation,true,"Map(numDeletedFiles -> 0, numVacuumedDirectories -> 1)",null,Databricks-Runtime/13.3.x-scala2.12
1,2023-12-13T11:28:07Z,1311347395691033,totnvt02@gmail.com,VACUUM START,"Map(retentionCheckEnabled -> true, defaultRetentionMillis -> 604800000)",null,List(1731309032100310),1122-230829-ksqo7xkx,0,SnapshotIsolation,true,"Map(numFilesToDelete -> 0, sizeOfDataToDelete -> 0)",null,Databricks-Runtime/13.3.x-scala2.12
0,2023-12-13T11:27:52Z,1311347395691033,totnvt02@gmail.com,WRITE,"Map(mode -> Overwrite, statsOnLoad -> false, partitionBy -> [])",null,List(1731309032100310),1122-230829-ksqo7xkx,null,WriteSerializable,false,"Map(numFiles -> 4, numOutputRows -> 87944, numOutputBytes -> 2052911)",null,Databricks-Runtime/13.3.x-scala2.12


In [0]:

%sql
select * from silver.keyword

movieId,keywords
1,"List(time portal, marriage for money, love of one's life, duke, advertising executive, gentleman, stage actor, advertising agency, scientist, brooklyn bridge, fish out of water, new york city, time deadline, marry rich, time travel)"
12,"List(night life, lovesickness, fountain, cowardliness, photographer, ghost, sadness, unsociability, dying and death, newspaper, loss of loved one, rome, italy, sunrise, sea)"
13,"List(fraud, con man, religious hypocrisy, traveling salesman, revivalism, kansas, usa, evangelist, religion, based on novel or book, preacher)"
22,"List(cinematographer, illegal prostitution, photography, proto-slasher)"
26,"List(milan, work, working class, matriarch, boxer, illegal prostitution, widow, murder, love, prostitute, parent child relationship, sibling relationship)"
27,"List(insurgence, escaped slave, gay interest, slavery, historical fiction, 1st century, ancient world, torture, criterion, roman empire, epic, ancient rome, technicolor, gladiator, gladiator fight, slave)"
28,"List(island, tree house, based on novel or book, pirate, ostrich, shipwreck, pirate gang, zebra, 19th century, swiss family robinson)"
31,"List(cigarette, writer, jewelry store, free spirit, broken engagement, millionaire, older woman younger man relationship, cat, kept man, best friends, store window, rich woman, new york city, love of one's life, gold digger)"
34,"List(global warming, heat, newspaper, earth, end of the world, sun, london, england, disaster, disaster movie)"
44,"List(puppy, dog, female villain, london, england, pets, dalmatian)"
